In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import nltk
import re

Eventually create a list of major codes and run a for loop to get all dataframes

In [58]:
def extract_course_catalog_info(dept): #where dept is a string
    site_url = 'https://www.ucsd.edu/catalog/courses/' + dept + '.html'
    page = requests.get(site_url)
    soup = BeautifulSoup(page.content, 'html.parser')
    

In [2]:
#site_url = 'https://www.ucsd.edu/catalog/courses/' + 'dept' + '.html'

site_url = 'https://www.ucsd.edu/catalog/courses/MATH.html'
dept = 'Math'
page = requests.get(site_url)
soup = BeautifulSoup(page.content, 'html.parser')

#https://stackoverflow.com/questions/49010813/how-to-find-a-particular-class-in-beautifulsoup-using-findall

## objctv: With soup, slice between each anchor parent, to find classes without prerequisites

# This is making page source into a string.  Try to find what delineates each element

In [3]:
page_source = []
for i in soup:
    page_source = np.append(page_source, str(i))

In [4]:
page_text0 = soup.get_text()
page_text_array = np.array(re.findall(r'\w+', page_text0)) # Does not have punctuation marks!!

# Looking at just names and descriptions w/ htmlclass separation

In [5]:
#full_list = soup.find_all("p", {"class": "anchor-parent"}) 

names = soup.find_all("p", {"class": "course-name"})
descriptions = soup.find_all("p", {"class": "course-descriptions"}) #includes prereqs (so want to extract that)


In [6]:
name_list = [] #this is in HTML contains all course header info

for x in names:
    name_list.append(str(x))

In [7]:
course_ids = []
course_names = []
course_units = [] 

for n in np.arange(len(name_list) - 1):
    #for i in np.arange(len(name_list[n]) - 1):
    x = name_list[n]
    course_ids.append(x[len('<p class="course-name">') + len(dept) + 1 : x.index('.')])
    course_names.append(x[x.index('.')+2:len(x)-8])
    course_units.append(x[len(x)-6])

## Some course names are weird (why?) e.g. 'Foundations\n\t\t  of Teaching and Learning Mathematics I'

## Course units cut off (why?)

In [8]:
course_nums = []

for i in course_ids:
    course_nums.append(re.findall(r'\d+|\D+', i)) #regex makes id's in format 'Num' + 'Letter'    

#regex for this \d+|\D+

# just nums: r'^\d+'

#### use list comprehension it is cleaner

finding indices of class category

In [9]:


for i in course_nums:
    if len(i[0]) == 2:
        try:
            lower_div_i = course_ids.index(i[0]+i[1]) #last index of lower divisions
        except IndexError:
            lower_div_i = course_ids.index(i[0])
    if int(i[0]) < 200:
        try:
            upper_div_i = course_ids.index(i[0]+i[1]) #" " of upper divisions
        except IndexError:
            upper_div_i = course_ids.index(i[0])
        
#[i for i,val in enumerate(l) if val==]

https://stackoverflow.com/questions/6294179/how-to-find-all-occurrences-of-an-element-in-a-list


### How to find last value before 200

In [10]:
#converting description blocks into lists

desc_list = []
for i in descriptions:
    desc_list.append(re.findall(r"\w+,*", str(i)))

In [11]:
# Removing graduate courses

course_ids = course_ids[0:upper_div_i + 1]
course_names = course_names[0:upper_div_i + 1]
course_units = course_units[0:upper_div_i + 1]
desc_list = desc_list[0:upper_div_i + 1]

In [12]:
#extracting prerequisites.  

prereq_list = []

for i in np.arange(len(desc_list)):
    x = desc_list[i]
    try: 
        prereq_list.append(x[x.index('Prerequisites')+2:len(x)-1])
    except ValueError:
        print('prereq extraction error at index', i) 
        prereq_list.append('None')

In [37]:
prereq_list[25]

['MATH', '31CH', 'or', 'MATH', '109', 'or', 'consent', 'of', 'instructor']

## Check where the ValueErrors occur!  In some cases it may be faulty html e.g Math 157 has weird indent and misses prereq section

Before cell below, need to clear the following words/strings: 
'either'

and turn this:

'Students',
  'who',
  'have',
  'not',
  'completed',
  'listed',
  'prerequisites',
  'may',
  'enroll',
  'with',
  'consent',
  'of',
  'instructor'
  
into just " or + consent of instructor"


### might have to parse for or and make the lists 3d

In [49]:
prereqs_2d = []
fuckyou = []
for n in np.arange(len(course_ids)):
    ps = prereq_list[n]
    ps_2d = [[]]
    inner_i = 0
    for i in np.arange(len(ps)):    
        if ps[i] != 'and':
            ps_2d[inner_i].append(ps[i])
        else:
            ps_2d.append([])
            inner_i = inner_i + 1
    prereqs_2d.append(ps_2d)

In [54]:
course_ids.index('180A')

75

In [56]:
prereqs_2d[76]

[['MATH', '20D'],
 ['either', 'MATH', '18', 'or', 'MATH', '20F', 'or', 'MATH', '31AH,'],
 ['MATH', '109', 'or', 'MATH', '31CH,'],
 ['MATH',
  '180A',
  'Students',
  'who',
  'have',
  'not',
  'completed',
  'listed',
  'prerequisites',
  'may',
  'enroll',
  'with',
  'consent',
  'of',
  'instructor']]

Below are two ways to parse "and".  Use second one

In [2]:
 #['MATH', '31CH', 'or', 'MATH', '109', 'or', 'consent', 'of', 'instructor']

#['A', '1', 'or', 'A', '2', 'and', 'A', '5', 'and', 'A', '6', 'or', 'A', '7', 'and', 'A', '9']
ps_2d =  []

parse_i = 0

for i in np.arange(len(ps) - 1):    
    if ps[i] == 'and':
        ps_2d.append(list(ps[ parse_i : i]) )
        parse_i = i +1
ps_2d.append(list(ps[ parse_i:]) ) # <<< wat??

In [45]:
ps = ['A', '1', 'or', 'A', '2', 'and', 'A', '5', 'and', 'A', '6', 'or', 'A', '7', 'and', 'A', '9']
ps_2d =  [[]]

inner_i = 0

for i in np.arange(len(ps)):    
    if ps[i] != 'and':
        ps_2d[inner_i].append(ps[i])
    else:
        ps_2d.append([])
        inner_i = inner_i + 1


all classes are going to have to be in the same object unless more looping

In [57]:
for prereq in prereqs_for_some_class:
    preprereqs = [] # prereqs for this prereq 
    look at prereqs of the prereq:
        repeat?


for i in prereqs:
    prereqs_3d[course_ids.index(i)]
    

# For prereqs, how to separate or, and etc

In [371]:
alt_prereq = 'consent of instructor'

# Want to create dataframe with Columns: name, ID, prereqs, descs (?)

In [ ]:
course_info_df = pd.DataFrame({'Course id': course_ids, 'Course name': course_names, 'Course units' = course_units, 'Prereqs' = })

Course names array must be in 2d.

[[Math 2, Introduction to College Mathematics], [MATH 3, Precalculus] ...] 

In [78]:

#course_page = course_page[0:int(np.where(course_page == '<p class="course-name">MATH 200A-B-C. Algebra (4-4-4)</p>')[0])]

(205,)


In [51]:
courses = []
start = ' <p class="course-name"> '
stop = '</p>'
for i in course_page:
    for s in i:
        

SyntaxError: unexpected EOF while parsing (<ipython-input-51-89e5bcb3f799>, line 5)

# once course list is isolated, find where graduate courses (>199) begin to slice 

# GRAPHS

adjacency matrix or array of vertex pairs?

-adj lists

have to consider inter departmental case and how to access their respective course_ids etc

The loop should keep going as long as it can find the class in a prereq object.

dictionaries?  [Dept, ID] : [prereqs]

In [ ]:
eg_prq = [['CSE 8B', 'CSE 11'],  ['MATH 18', 'MATH 31AH'], ['MATH 20E'], ['MATH 180A']]



for i in eg_prq:
    for n in eg_prq[i]:
        #dept = n[0][0]
                
                if eg_prq[i] == prereq_list[x] 